#### 3.1 Supervised DA

##### 3.1.1 In-domain fine-tuning

Подготовка данных

In [1]:
import datasets
import typing as tp
import inspect

conll = datasets.load_dataset("conll2003") # DatasetDict
wnut = datasets.load_dataset("wnut_17")

conll.save_to_disk("datasets/conll2003")
wnut.save_to_disk("datasets/wnut_17")

CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names # всего 9 тегов
WNUT_NER_TAGS = wnut['train'].features['ner_tags'].feature.names

/home/daniil_tomlv/miniconda3/envs/lmenv/lib/python3.10/importlib/__init__.py:126: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.22.3)
  return _bootstrap._gcd_import(name[level:], package, level)


Saving the dataset (0/1 shards):   0%|          | 0/14041 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3250 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3453 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3394 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1009 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1287 [00:00<?, ? examples/s]

In [2]:
conll

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
wnut

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [4]:
CONLL_NER_TAGS

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [5]:
WNUT_NER_TAGS

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [7]:
label_mapping = {
    'O': 'O',
    'B-location': 'B-LOC',
    'I-location': 'I-LOC',
    'B-group': 'B-ORG',
    'B-corporation': 'B-ORG',
    'B-person': 'B-PER',
    'B-creative-work': 'B-MISC',
    'B-product': 'B-MISC',
    'I-person': 'I-PER',
    'I-creative-work': 'I-MISC',
    'I-corporation': 'I-ORG',
    'I-group': 'I-ORG',
    'I-product': 'I-MISC'
}

labelindexmapping = {WNUT_NER_TAGS.index(k):CONLL_NER_TAGS.index(v) for k, v in label_mapping.items()}

import data_prep

converted_wnut = wnut.map(lambda x: data_prep.convert_label_sequence(x, labelindexmapping))

import numpy as np
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline

import torch 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from tqdm.auto import tqdm

from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

model_name = "dslim/bert-base-NER"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
from datasets import load_dataset
import torch
from tqdm.auto import tqdm
from transformers import AdamW, BertTokenizer, BertForMaskedLM, set_seed
import pandas as pd
import warnings

set_seed(42)
warnings.filterwarnings("ignore")
import data_prep
import importlib
importlib.reload(data_prep)

label2id_conll= {'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8
}

id2label_conll = {v : k for k, v in label2id_conll.items()}

test_sentence = "His name is Jerry Abrahamson"
test_example = {"tokens": test_sentence.split(" "), "ner_tags": [0, 0, 0, 1, 2]}
# test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, model.config.label2id)
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)

assert tokenizer.decode(test_result['input_ids']) == '[CLS] His name is Jerry Abrahamson [SEP]'

                                     #CLS     His  name is    Jerry    Abraham   ##son      SEP
assert test_result['text_labels'] == ['O'] + ["O", "O", "O", "B-PER", "I-PER",  "I-PER"] + ["O"]

conll = conll.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

wnut = converted_wnut
wnut = wnut.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

conll.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)
wnut.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
import importlib
import data_prep
importlib.reload(data_prep)

test_sentence = "His name is Jerry Abrahamson"
test_example = {"tokens": test_sentence.split(" "), "ner_tags": [0, 0, 0, 1, 2]}
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)
print(test_result.word_ids())
print(test_result['input_ids'])
print([tokenizer.decode(token) for token in test_result['input_ids']])

[None, 0, 1, 2, 3, 4, 4, None]
[101, 1230, 1271, 1110, 5466, 7752, 2142, 102]
['[CLS]', 'His', 'name', 'is', 'Jerry', 'Abraham', '##son', '[SEP]']


In [9]:
test_result

{'input_ids': [101, 1230, 1271, 1110, 5466, 7752, 2142, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'tokens': ['His', 'name', 'is', 'Jerry', 'Abrahamson'], 'ner_tags': [0, 0, 0, 1, 2], 'labels': [0, 0, 0, 0, 1, 2, 2, 0], 'text_labels': ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O']}

Теперь есть готовые датасеты conll и wnut

Создадим даталоадеры для трейн и теста conll и wnut

In [10]:
input_keys = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
batch_size = 16
wnut_train_dataloader = torch.utils.data.DataLoader(wnut["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
wnut_test_dataloader = torch.utils.data.DataLoader(wnut["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_train_dataloader = torch.utils.data.DataLoader(conll["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_test_dataloader = torch.utils.data.DataLoader(conll["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))


In [11]:
batch=next(iter(wnut_test_dataloader))
print(len(batch['ner_tags'][7]))
print(len(batch['input_ids'][7]))
print(len(batch['tokens'][7]))
print(len(batch['labels'][7]))

14
45
14
45


In [12]:
test_batch = {k : [v] for k, v in test_result.items()}

Измерим метрики на базовой версии BERTа

In [13]:
model_name = "dslim/bert-base-NER" # дообучена на датасете CoNLL-2003
model_base = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_base.to(device)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [14]:
test_batch

{'input_ids': [[101, 1230, 1271, 1110, 5466, 7752, 2142, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]],
 'tokens': [['His', 'name', 'is', 'Jerry', 'Abrahamson']],
 'ner_tags': [[0, 0, 0, 1, 2]],
 'labels': [[0, 0, 0, 0, 1, 2, 2, 0]],
 'text_labels': [['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O']]}

In [15]:
# тест, что все адекватно предсказывается
import torch
from collections import defaultdict

batched_test_batch = defaultdict(list)

for key, value in test_batch.items():
    try:
        batched_test_batch[key] = torch.tensor(value)
    except ValueError:
        batched_test_batch[key] = value

for key in batched_test_batch:
    if isinstance(batched_test_batch[key], torch.Tensor):
        batched_test_batch[key] = batched_test_batch[key].to(device)

batched_test_batch

defaultdict(list,
            {'input_ids': tensor([[ 101, 1230, 1271, 1110, 5466, 7752, 2142,  102]], device='cuda:0'),
             'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
             'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'),
             'tokens': [['His', 'name', 'is', 'Jerry', 'Abrahamson']],
             'ner_tags': tensor([[0, 0, 0, 1, 2]], device='cuda:0'),
             'labels': tensor([[0, 0, 0, 0, 1, 2, 2, 0]], device='cuda:0'),
             'text_labels': [['O',
               'O',
               'O',
               'O',
               'B-PER',
               'I-PER',
               'I-PER',
               'O']]})

In [16]:
model_base.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [17]:
id2label_conll

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [18]:
import importlib
import model_utils
importlib.reload(model_utils)

from model_utils import NamedEntityPredictor

ner_test = NamedEntityPredictor(model_base, tokenizer, id2label_conll)
ner_test.predict(batched_test_batch)

{'predicted_labels': [['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O']],
 'loss': tensor(3.5653, device='cuda:0'),
 'logits': tensor([[[ 6.8225, -0.2862, -1.0980, -0.3375, -1.1505, -1.0953, -1.7006,
           -0.8111, -1.3737],
          [ 9.3304, -1.1577, -2.1227, -0.1745, -1.8043, -1.7342, -2.2795,
           -0.7647, -1.3315],
          [ 9.7455, -1.0748, -1.7252, -0.2097, -1.3579, -2.4633, -1.7645,
           -1.0774, -1.2969],
          [ 9.3520, -1.2463, -1.9811, -0.2193, -1.5422, -2.2553, -1.8455,
           -0.9268, -1.1779],
          [-0.8416, -1.2910, -2.3588,  8.7858, -0.5230, -1.6059, -2.6656,
           -0.3291, -1.6418],
          [-0.7083, -1.2668, -1.3766, -1.0028,  8.9504, -1.9978, -0.7555,
           -0.9198, -1.0510],
          [ 1.7984, -1.4755, -1.4079, -0.3893,  6.5783, -2.9142, -1.2547,
           -0.7049, -1.4439],
          [ 4.4946, -0.4864, -1.3231,  1.8384,  0.9670, -3.0517, -1.8221,
           -0.5035, -1.0015]]], device='cuda:0')}

In [19]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, get_scheduler
import torch
from torch.nn.utils import clip_grad_norm_ # выполняет обрезку градиентов для избежания их взрыва при обучении
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

import importlib
import data_prep
import model_utils
importlib.reload(data_prep)
importlib.reload(model_utils)

from model_utils import train_eval_ner

Файн-тюн сразу всего BERTа

In [20]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
optimizer = torch.optim.AdamW(trainable_params, lr=1e-5, weight_decay=0.01)

num_epochs = 2
warmup_factor = 0.1

ft_train_dataset = wnut['train']
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 40.45it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:14<00:00, 14.69it/s]


Epoch: 1	 Train: 0.10427977619561511 	 Test: 0.33142581998290105
F1: 0.3707802988378528
LR: [5.546875e-06]



Epoch 2: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:14<00:00, 14.60it/s]


Epoch: 2	 Train: 0.09019475296454531 	 Test: 0.32817376322216457
F1: 0.3867521367521367
LR: [0.0]

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.88 (+0.13)  0.91 (+0.07)  0.89 (+0.10)  0.55 (+0.10)   
MISC          0.64 (+0.06)  0.66 (-0.04)  0.65 (+0.02)  0.24 (+0.07)   
ORG           0.84 (+0.13)  0.87 (+0.05)  0.86 (+0.10)  0.32 (+0.16)   
PER           0.79 (+0.43)  0.92 (+0.34)  0.85 (+0.40)  0.65 (+0.35)   
micro avg     0.81 (+0.23)  0.87 (+0.13)  0.84 (+0.19)  0.46 (+0.22)   
macro avg     0.79 (+0.19)  0.84 (+0.10)  0.81 (+0.15)  0.44 (+0.17)   
weighted avg  0.81 (+0.20)  0.87 (+0.13)  0.84 (+0.18)  0.46 (+0.20)   

                    rec_ft         f1_ft  
LOC           0.47 (-0.04)  0.51 (+0.03)  
MISC          0.17 (-0.05)  0.20 (+0.01)  
ORG           0.26 (-0.04)  0.28 (+0.07)  
PER           0.43 (+0.13)  0.52 (+0.22)  
micro avg     0.34 (+0.03)  0.39 (+0.12)  
macro avg     0.33 (+0.00)  0.38 (+0.09)  
weighted avg  0.34 (+0.

Если файн-тюнить только классификатор

In [25]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
# optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
optimizer = torch.optim.Adam(trainable_params, lr=5e-4, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = wnut['train']
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

# lr_scheduler = get_scheduler(
#     name="linear",
#     optimizer=optimizer,
#     num_warmup_steps=int(warmup_factor * num_training_steps),
#     num_training_steps=num_training_steps,
# )

lr_scheduler = get_scheduler(
    name="constant",
    optimizer=optimizer
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 39.80it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:03<00:00, 53.93it/s]


...

Epoch: 8	 Train: 0.12209231433559192 	 Test: 0.2898045534980886
F1: 0.35334476843910806
LR: [0.0005]



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:03<00:00, 53.86it/s]


Epoch: 9	 Train: 0.12107299114774231 	 Test: 0.2939505698964184
F1: 0.3502304147465437
LR: [0.0005]



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 213/213 [00:03<00:00, 53.77it/s]


Epoch: 10	 Train: 0.11958555796756432 	 Test: 0.2915529474891998
F1: 0.35079726651480636
LR: [0.0005]

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.81 (+0.06)  0.87 (+0.03)  0.84 (+0.05)  0.57 (+0.12)   
MISC          0.15 (-0.43)  0.01 (-0.69)  0.01 (-0.62)  0.05 (-0.12)   
ORG           0.67 (-0.04)  0.73 (-0.09)  0.70 (-0.06)  0.29 (+0.13)   
PER           0.68 (+0.32)  0.82 (+0.24)  0.74 (+0.29)  0.60 (+0.30)   
micro avg     0.72 (+0.14)  0.71 (-0.03)  0.71 (+0.06)  0.45 (+0.21)   
macro avg     0.58 (-0.02)  0.61 (-0.13)  0.57 (-0.09)  0.38 (+0.11)   
weighted avg  0.65 (+0.04)  0.71 (-0.03)  0.67 (+0.01)  0.40 (+0.14)   

                    rec_ft         f1_ft  
LOC           0.47 (-0.04)  0.52 (+0.04)  
MISC          0.01 (-0.21)  0.02 (-0.17)  
ORG           0.23 (-0.07)  0.26 (+0.05)  
PER           0.42 (+0.12)  0.50 (+0.20)  
micro avg     0.29 (-0.02)  0.35 (+0.08)  
macro avg     0.28 (-0.05)  0.32 (+0.03)  
weighted avg  0.29 

##### 3.1.2  Resampling methods

Отберем топ 25% примеров из conll по схожести с wnut с помощью бинарного классификатора, затем сделаем fine-tuning модели на этих топ 25 % примерах

In [26]:
from datasets import concatenate_datasets
importlib.reload(model_utils)

columns = ["input_ids", "token_type_ids", "attention_mask", "labels", "domain_label"]

conll_labeled = conll["train"].map(lambda x: {"domain_label": int(0)})
columns_to_remove = [col for col in conll_labeled.column_names if col not in columns]
conll_labeled = conll_labeled.remove_columns(columns_to_remove)

wnut_labeled = wnut["train"].map(lambda x: {"domain_label": int(1)})
columns_to_remove = [col for col in wnut_labeled.column_names if col not in columns]
wnut_labeled = wnut_labeled.remove_columns(columns_to_remove)

combined = concatenate_datasets([conll_labeled, wnut_labeled]) 

In [27]:
# делим combined на k фолдов, учим k классификаторов на остальных k-1 фолдах, добавляем в combined wnut_score
# для каждого примера получаем эмбеддинг предложения с помощью [CLF] токена BERTа
# для каждого примера из тестового фолда добавляем wnut_score
# затем из combined отбираем топ 25 % примеров по этому скору

# сначала получить эмбеддинги всех предложений
# затем сделать k классификаторов по типу leave-one-out
# классификатор X: [batch_size, emb_len] -> wnut_scores: [batch_size]

importlib.reload(model_utils)
importlib.reload(data_prep)

model_name = "dslim/bert-base-NER"
model = AutoModelForTokenClassification.from_pretrained(model_name)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataloader = torch.utils.data.DataLoader(
    combined,
    batch_size=32,
    collate_fn=data_prep.PadSequence(['input_ids', 'token_type_ids', 'attention_mask', 'labels']),
)

all_embeddings = []

for batch in tqdm(dataloader, desc="Extracting embeddings"):
    embeddings = model_utils.get_sentence_embeddings(model, batch)  # [batch_size, emb_size]
    all_embeddings.append(embeddings)

all_embeddings = np.vstack(all_embeddings)  # [num_rows, emb_size]

combined = combined.add_column("cls_embedding", all_embeddings.tolist())

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Extracting embeddings: 100%|██████████████████████████████████████████████████████████| 545/545 [00:18<00:00, 28.79it/s]


In [28]:
combined

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'domain_label', 'cls_embedding'],
    num_rows: 17435
})

In [29]:
combined['cls_embedding'].shape

torch.Size([17435, 768])

In [30]:
# теперь добавим wnut_score в combined для дальнейшего отбора

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
wnut_scores = np.zeros(len(combined))

indices = np.arange(len(combined))
for train_idxs, test_idxs in kf.split(indices):
    train_examples = combined.select(train_idxs)
    test_examples = combined.select(test_idxs)

    train_dataloader = torch.utils.data.DataLoader(
        train_examples, batch_size=32,
        collate_fn=data_prep.PadSequence(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    )
    test_dataloader = torch.utils.data.DataLoader(
        test_examples, batch_size=32,
        collate_fn=data_prep.PadSequence(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    )

    X_train, Y_train = [], []

    for batch in tqdm(train_dataloader, desc="Train embeddings"):
        embs = torch.stack(batch["cls_embedding"]).cpu().numpy()
        domains = torch.stack(batch['domain_label']).cpu().numpy()
        X_train.append(embs)
        Y_train.append(domains)
    X_train = np.concatenate(X_train)
    Y_train = np.concatenate(Y_train)

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, Y_train)

    X_test = []
    for batch in tqdm(test_dataloader, desc="Test embeddings"):
        embs = torch.stack(batch["cls_embedding"]).cpu().numpy()
        X_test.append(embs)
    X_test = np.concatenate(X_test)
    probas = clf.predict_proba(X_test)[:, 1]

    wnut_scores[test_idxs] = probas

combined = combined.add_column("wnut_score", wnut_scores.tolist())


Test embeddings: 100%|███████████████████████████████████████████████████████████████| 109/109 [00:00<00:00, 239.40it/s]


In [31]:
combined

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'domain_label', 'cls_embedding', 'wnut_score'],
    num_rows: 17435
})

In [32]:
# отбираем топ 25 % объектов из conll по схожести с датасетом с wnut

conll_subset = combined.filter(lambda x: x["domain_label"] == 0)

conll_sorted = conll_subset.sort("wnut_score", reverse=True) 

# построить гистограмму с wnut_score

top_25_count = int(0.25 * len(conll_sorted))
top_25_conll = conll_sorted.select(range(top_25_count))

top_25_conll = top_25_conll.remove_columns([
    col for col in top_25_conll.column_names if col not in ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
])

Filter:   0%|          | 0/17435 [00:00<?, ? examples/s]

Теперь есть топ 25 % примеров из CONLL для файн-тюна BERTа

In [33]:
top_25_conll

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3510
})

In [34]:
wnut['train']

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'text_labels'],
    num_rows: 3394
})

Размеры датасетов wnut['train'] и top_25_conll почти одинаковы. Посмотрим что будет, если файн-тюнить классификатор на top_25_conll с теми же гиперпараметрами, что и для файн-тюна на wnut['train']

In [35]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
# optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
optimizer = torch.optim.Adam(trainable_params, lr=5e-4, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = top_25_conll
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="constant",
    optimizer=optimizer
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 40.09it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 220/220 [00:03<00:00, 61.48it/s]


...

Epoch: 8	 Train: 0.02209912260084159 	 Test: 0.41661112456593985
F1: 0.33042789223454827
LR: [0.0005]



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 220/220 [00:03<00:00, 61.10it/s]


Epoch: 9	 Train: 0.021720438595564866 	 Test: 0.4194184423964701
F1: 0.3280632411067194
LR: [0.0005]



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 220/220 [00:03<00:00, 60.94it/s]


Epoch: 10	 Train: 0.020791363586216572 	 Test: 0.41656336183717224
F1: 0.32986389111289033
LR: [0.0005]

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.89 (+0.14)  0.92 (+0.08)  0.90 (+0.11)  0.40 (-0.05)   
MISC          0.64 (+0.06)  0.75 (+0.05)  0.69 (+0.06)  0.13 (-0.04)   
ORG           0.83 (+0.12)  0.90 (+0.08)  0.86 (+0.10)  0.15 (-0.01)   
PER           0.83 (+0.47)  0.93 (+0.35)  0.88 (+0.43)  0.61 (+0.31)   
micro avg     0.82 (+0.24)  0.89 (+0.15)  0.86 (+0.21)  0.29 (+0.05)   
macro avg     0.80 (+0.20)  0.87 (+0.13)  0.83 (+0.17)  0.32 (+0.05)   
weighted avg  0.82 (+0.21)  0.89 (+0.15)  0.86 (+0.20)  0.36 (+0.10)   

                    rec_ft         f1_ft  
LOC           0.49 (-0.02)  0.44 (-0.04)  
MISC          0.20 (-0.02)  0.16 (-0.03)  
ORG           0.32 (+0.02)  0.21 (+0.00)  
PER           0.49 (+0.19)  0.54 (+0.24)  
micro avg     0.38 (+0.07)  0.33 (+0.06)  
macro avg     0.38 (+0.05)  0.34 (+0.05)  
weighted avg  0.3

#### 3.2 Unsupervised DA

##### 3.2.1 Proxy-labels methods

До этого использовались размеченные данные. Теперь будем отбирать наиболее уверенные с точки зрения пре-тренированной модели примеры, делать предсказания и дообучать на этих прокси-лейблах, чтобы перенести знания из целевого домена.

In [36]:
# лучше сделать рестарт ядра перед запуском ячеек ниже
# ниже повторная предобработка данных
import datasets
import typing as tp
import inspect

conll = datasets.load_dataset("conll2003") # DatasetDict
wnut = datasets.load_dataset("wnut_17")

CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names # всего 9 тегов
WNUT_NER_TAGS = wnut['train'].features['ner_tags'].feature.names

label_mapping = {
    'O': 'O',
    'B-location': 'B-LOC',
    'I-location': 'I-LOC',
    'B-group': 'B-ORG',
    'B-corporation': 'B-ORG',
    'B-person': 'B-PER',
    'B-creative-work': 'B-MISC',
    'B-product': 'B-MISC',
    'I-person': 'I-PER',
    'I-creative-work': 'I-MISC',
    'I-corporation': 'I-ORG',
    'I-group': 'I-ORG',
    'I-product': 'I-MISC'
}

label2id_conll= {'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

labelindexmapping = {WNUT_NER_TAGS.index(k):CONLL_NER_TAGS.index(v) for k, v in label_mapping.items()}

import data_prep

converted_wnut = wnut.map(lambda x: data_prep.convert_label_sequence(x, labelindexmapping))

import numpy as np
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline

import torch 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from tqdm.auto import tqdm

from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)

import importlib
import data_prep

importlib.reload(data_prep)

test_sentence = "His name is Jerry Abrahamson"
test_example = {"tokens": test_sentence.split(" "), "ner_tags": [0, 0, 0, 1, 2]}
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)

assert tokenizer.decode(test_result['input_ids']) == '[CLS] His name is Jerry Abrahamson [SEP]'

                                     #CLS     His  name is    Jerry    Abraham   ##son      SEP
assert test_result['text_labels'] == ['O'] + ["O", "O", "O", "B-PER", "I-PER",  "I-PER"] + ["O"]

conll = conll.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

wnut = converted_wnut
wnut = wnut.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

conll.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)
wnut.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)

input_keys = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
batch_size = 32
wnut_train_dataloader = torch.utils.data.DataLoader(wnut["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
wnut_test_dataloader = torch.utils.data.DataLoader(wnut["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_train_dataloader = torch.utils.data.DataLoader(conll["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_test_dataloader = torch.utils.data.DataLoader(conll["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))

In [37]:
model_name = "dslim/bert-base-NER"
model = AutoModelForTokenClassification.from_pretrained(model_name)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
batch = next(iter(wnut_train_dataloader))
with torch.no_grad():
    outputs = model(input_ids=batch["input_ids"],
                 token_type_ids=batch["token_type_ids"],
                 attention_mask=batch["attention_mask"],
                 labels=batch["labels"], return_dict=True)

print(outputs.logits.shape)
attn_mask = batch['attention_mask']
probs = torch.softmax(outputs.logits, dim=-1)
max_probs = probs.max(dim=-1).values

confidences = []
for i in range(max_probs.shape[0]):
    mask = attn_mask[i].bool()
    conf = max_probs[i][mask].mean().item()
    confidences.append(conf)

print(confidences)

torch.Size([32, 47, 9])
[0.9770016074180603, 0.9972400665283203, 0.9488641023635864, 0.9996575713157654, 0.9515469074249268, 0.9970041513442993, 0.999862551689148, 0.999904215335846, 0.9905007481575012, 0.9993622303009033, 0.989292562007904, 0.9799342751502991, 0.9869406819343567, 0.9998921751976013, 0.9998874068260193, 0.9990612864494324, 0.957036018371582, 0.9998361468315125, 0.9946780204772949, 0.9996030926704407, 0.9914761185646057, 0.9993963241577148, 0.9936321377754211, 0.999690592288971, 0.999440610408783, 0.9997891187667847, 0.9997739791870117, 0.9849512577056885, 0.9998698234558105, 0.9916747808456421, 0.9861404895782471, 0.9998873472213745]


In [39]:
from tqdm.auto import tqdm

model = model.to(device)
model.eval()

all_confidences = []

with torch.no_grad():
    for batch in tqdm(wnut_train_dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.softmax(outputs.logits, dim=-1)             # [B, T, C]
        max_probs = probs.max(dim=-1).values                      # [B, T]

        for i in range(input_ids.size(0)):
            real_token_mask = attention_mask[i].bool()            # [T]
            conf = max_probs[i][real_token_mask].mean().item()
            all_confidences.append(conf)

  0%|          | 0/107 [00:00<?, ?it/s]

In [40]:
# убедимся, что длина совпадает
assert len(all_confidences) == len(wnut["train"])

# добавим как новое поле
wnut["train"] = wnut["train"].add_column("confidence", all_confidences)

In [41]:
confidences = np.array(all_confidences)
threshold = np.percentile(confidences, 50)

filtered_wnut = wnut["train"].filter(lambda example: example["confidence"] >= threshold)

Filter:   0%|          | 0/3394 [00:00<?, ? examples/s]

Датасет для файн-тюна готов.

Как обычно загрузим базовую модель (без файн-тюна) и будем сравнивать результаты с ней

In [42]:
filtered_wnut

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'text_labels', 'confidence'],
    num_rows: 1697
})

In [43]:
# добавляем прокси-метки в датасет
filtered_wnut_dataloader = torch.utils.data.DataLoader(filtered_wnut,        
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))

# снова получим прокси-лейблы для топ 50 процентов самых уверенных примеров из wnut_train
# получаем прокси-лейблы
ner = NamedEntityPredictor(model_ft, tokenizer, id2label=id2label_conll)
text_proxy_labels = []
proxy_labels = []

for batch in filtered_wnut_dataloader:
    text_labels_batch = ner.predict(batch)['predicted_labels']  # list[list[str]]
    text_proxy_labels.extend(text_labels_batch)

    # переводим текстовые метки в индексы
    index_batch = [
        [label2id_conll[label] for label in seq]
        for seq in text_labels_batch
    ]
    proxy_labels.extend(index_batch)

# создаем новый датасет с заменой labels и text_labels
filtered_wnut_proxy = filtered_wnut.remove_columns(['labels', 'text_labels']) \
                                   .add_column('labels', proxy_labels) \
                                   .add_column('text_labels', text_proxy_labels)

Flattening the indices:   0%|          | 0/1697 [00:00<?, ? examples/s]

In [45]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False

# for name, param in model_ft.named_parameters():
#     if name.startswith("bert.embeddings") or name.startswith("bert.encoder.layer.0") or \
#        name.startswith("bert.encoder.layer.1") or name.startswith("bert.encoder.layer.2") or \
#        name.startswith("bert.encoder.layer.3") or name.startswith("bert.encoder.layer.4") or \
#        name.startswith("bert.encoder.layer.5") or name.startswith("bert.encoder.layer.6") or \
#        name.startswith("bert.encoder.layer.7"):
#         param.requires_grad = False
    
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
# optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
optimizer = torch.optim.Adam(trainable_params, lr=1e-4, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = filtered_wnut_proxy
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="constant",
    optimizer=optimizer
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 39.93it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:01<00:00, 58.31it/s]


...

Epoch: 8	 Train: 0.003793370290104491 	 Test: 0.44196327267513597
F1: 0.33773987206823025
LR: [0.0001]



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:01<00:00, 58.93it/s]


Epoch: 9	 Train: 0.0036362559571601104 	 Test: 0.44582081190597866
F1: 0.341025641025641
LR: [0.0001]



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:01<00:00, 59.14it/s]


Epoch: 10	 Train: 0.003502310390222957 	 Test: 0.4455311949146383
F1: 0.3388676032354193
LR: [0.0001]

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.78 (+0.03)  0.85 (+0.01)  0.82 (+0.03)  0.50 (+0.05)   
MISC          0.60 (+0.02)  0.72 (+0.02)  0.65 (+0.02)  0.16 (-0.01)   
ORG           0.75 (+0.04)  0.85 (+0.03)  0.80 (+0.04)  0.17 (+0.01)   
PER           0.62 (+0.26)  0.82 (+0.24)  0.71 (+0.26)  0.55 (+0.25)   
micro avg     0.70 (+0.12)  0.82 (+0.08)  0.76 (+0.11)  0.31 (+0.07)   
macro avg     0.69 (+0.09)  0.81 (+0.07)  0.74 (+0.08)  0.34 (+0.07)   
weighted avg  0.71 (+0.10)  0.82 (+0.08)  0.76 (+0.10)  0.36 (+0.10)   

                    rec_ft         f1_ft  
LOC           0.52 (+0.01)  0.51 (+0.03)  
MISC          0.19 (-0.03)  0.17 (-0.02)  
ORG           0.32 (+0.02)  0.22 (+0.01)  
PER           0.46 (+0.16)  0.50 (+0.20)  
micro avg     0.37 (+0.06)  0.34 (+0.07)  
macro avg     0.37 (+0.04)  0.35 (+0.06)  
weighted avg  0.37 

Если заморозить первые 8 слоев BERTа

In [46]:
model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# for param in model_ft.bert.parameters():
#     param.requires_grad = False

for name, param in model_ft.named_parameters():
    if name.startswith("bert.embeddings") or name.startswith("bert.encoder.layer.0") or \
       name.startswith("bert.encoder.layer.1") or name.startswith("bert.encoder.layer.2") or \
       name.startswith("bert.encoder.layer.3") or name.startswith("bert.encoder.layer.4") or \
       name.startswith("bert.encoder.layer.5") or name.startswith("bert.encoder.layer.6") or \
       name.startswith("bert.encoder.layer.7"):
        param.requires_grad = False
    
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
optimizer = torch.optim.AdamW(trainable_params, lr=2e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = filtered_wnut_proxy
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 39.53it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:02<00:00, 37.13it/s]


...

Epoch: 8	 Train: 0.0007515545706232035 	 Test: 0.4748352336938734
F1: 0.3732394366197183
LR: [4.444444444444444e-06]



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:02<00:00, 37.75it/s]


Epoch: 9	 Train: 0.0006943032658965192 	 Test: 0.4771714671112505
F1: 0.3720316622691293
LR: [2.222222222222222e-06]



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:02<00:00, 37.54it/s]


Epoch: 10	 Train: 0.0006872540363695399 	 Test: 0.4766671623068827
F1: 0.3718285214348206
LR: [0.0]

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.81 (+0.06)  0.87 (+0.03)  0.84 (+0.05)  0.52 (+0.07)   
MISC          0.61 (+0.03)  0.72 (+0.02)  0.66 (+0.03)  0.19 (+0.02)   
ORG           0.81 (+0.10)  0.88 (+0.06)  0.84 (+0.08)  0.19 (+0.03)   
PER           0.74 (+0.38)  0.89 (+0.31)  0.81 (+0.36)  0.60 (+0.30)   
micro avg     0.76 (+0.18)  0.86 (+0.12)  0.81 (+0.16)  0.35 (+0.11)   
macro avg     0.74 (+0.14)  0.84 (+0.10)  0.79 (+0.13)  0.38 (+0.11)   
weighted avg  0.76 (+0.15)  0.86 (+0.12)  0.81 (+0.15)  0.40 (+0.14)   

                    rec_ft         f1_ft  
LOC           0.53 (+0.02)  0.52 (+0.04)  
MISC          0.22 (+0.00)  0.20 (+0.01)  
ORG           0.33 (+0.03)  0.25 (+0.04)  
PER           0.49 (+0.19)  0.54 (+0.24)  
micro avg     0.39 (+0.08)  0.37 (+0.10)  
macro avg     0.39 (+0.06)  0.38 (+0.09)  
weighted avg  0.39 (+

In [47]:
# сравним настоящие и прокси метки на одном примере
filtered_wnut_dataloader_proxy = torch.utils.data.DataLoader(filtered_wnut_proxy,        
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))

print(next(iter(filtered_wnut_dataloader))['labels'][0])
print(next(iter(filtered_wnut_dataloader_proxy))['labels'][0])

tensor([   0,    0,    0,    0,    0,    0,    0,    3,    4,    4,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100], device='cuda:0')
tensor([   0,    0,    3,    4,    4,    4,    0,    3,    4,    4,    0,    0,
           0,    7,    8,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100, -100,
        -100], device='cuda:0')


Сравнение с обычным файн-тюном на всем трейне wnut при тех же гиперпараметрах

In [50]:
# берем не топ 50 процентов по уверенности модели, а просто 50 процентов рандомных примеров из wnut train
import random

np.random.seed(42)
random_indices = random.sample(range(len(wnut["train"])), len(filtered_wnut))
wnut_train_random_subs = [wnut["train"][i] for i in random_indices]

model_name = "dslim/bert-base-NER"
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param in model_ft.bert.parameters():
    param.requires_grad = False

# for name, param in model_ft.named_parameters():
#     if name.startswith("bert.embeddings") or name.startswith("bert.encoder.layer.0") or \
#        name.startswith("bert.encoder.layer.1") or name.startswith("bert.encoder.layer.2") or \
#        name.startswith("bert.encoder.layer.3") or name.startswith("bert.encoder.layer.4") or \
#        name.startswith("bert.encoder.layer.5") or name.startswith("bert.encoder.layer.6") or \
#        name.startswith("bert.encoder.layer.7"):
#         param.requires_grad = False
    
trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
# optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
optimizer = torch.optim.Adam(trainable_params, lr=1e-4, betas=(0.9, 0.999), eps=1e-08)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = wnut_train_random_subs
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="constant",
    optimizer=optimizer
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)


fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 39.78it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:02<00:00, 52.85it/s]


...

Epoch: 8	 Train: 0.15525568419388522 	 Test: 0.28294920548796654
F1: 0.3437152391546162
LR: [0.0001]



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:01<00:00, 53.81it/s]


Epoch: 9	 Train: 0.15351878430381 	 Test: 0.2822234291169379
F1: 0.3479224376731302
LR: [0.0001]



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:01<00:00, 53.97it/s]


Epoch: 10	 Train: 0.1517657469261751 	 Test: 0.2816503979725602
F1: 0.34883720930232553
LR: [0.0001]

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.81 (+0.06)  0.88 (+0.04)  0.84 (+0.05)  0.55 (+0.10)   
MISC          0.62 (+0.04)  0.64 (-0.06)  0.63 (+0.00)  0.18 (+0.01)   
ORG           0.71 (+0.00)  0.75 (-0.07)  0.73 (-0.03)  0.35 (+0.19)   
PER           0.62 (+0.26)  0.78 (+0.20)  0.69 (+0.24)  0.56 (+0.26)   
micro avg     0.70 (+0.12)  0.78 (+0.04)  0.74 (+0.09)  0.43 (+0.19)   
macro avg     0.69 (+0.09)  0.76 (+0.02)  0.73 (+0.07)  0.41 (+0.14)   
weighted avg  0.70 (+0.09)  0.78 (+0.04)  0.74 (+0.08)  0.42 (+0.16)   

                    rec_ft         f1_ft  
LOC           0.49 (-0.02)  0.52 (+0.04)  
MISC          0.11 (-0.11)  0.14 (-0.05)  
ORG           0.24 (-0.06)  0.28 (+0.07)  
PER           0.37 (+0.07)  0.44 (+0.14)  
micro avg     0.29 (-0.02)  0.35 (+0.08)  
macro avg     0.30 (-0.03)  0.35 (+0.06)  
weighted avg  0.29 (

##### 3.2.2 Unsupervised pretraining


In [1]:
import json

data = []
with open('reddit_sample.json', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

from datasets import Dataset
reddit_dataset = Dataset.from_list(data)
reddit_dataset= reddit_dataset.rename_column("words", "tokens")

AttributeError: type object 'Dataset' has no attribute 'from_list'

Аналогично 3.1.2 отберем 100 000 примеров, наиболее похожих на wnut['train'] и дообучим BERT на задачу masked LM.

In [ ]:
wnut['train']

In [122]:
columns_to_keep = ['tokens'] # называется токены, но в датасете это просто списки слов
columns_to_remove = [col for col in wnut['train'].column_names if col not in columns_to_keep]

wnut_train_new = wnut['train'].remove_columns(columns_to_remove)

In [123]:
wnut_train_new

Dataset({
    features: ['tokens'],
    num_rows: 3394
})

In [124]:
reddit_dataset

Dataset({
    features: ['tokens'],
    num_rows: 500000
})

In [125]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

def tokenize_batch(batch):
    return tokenizer(
        batch["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding=False,
        max_length=512
    )

reddit_dataset = reddit_dataset.map(
    tokenize_batch,
    batched=True,
    remove_columns=[]
)

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [127]:
wnut_train_new = wnut_train_new.map(
    tokenize_batch,
    batched=True,
    remove_columns=[]
)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

In [128]:
reddit_dataset = reddit_dataset.map(lambda x: {"domain_label": int(0)})
wnut_train_new = wnut_train_new.map(lambda x: {"domain_label": int(1)})

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

In [129]:
reddit_dataset

Dataset({
    features: ['tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'domain_label'],
    num_rows: 500000
})

In [130]:
wnut_train_new

Dataset({
    features: ['tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'domain_label'],
    num_rows: 3394
})

In [131]:
from datasets import concatenate_datasets

combined = concatenate_datasets([reddit_dataset, wnut_train_new]) 

In [132]:
combined.set_format("torch")

In [134]:
combined['domain_label'][500_001]

tensor(1)

In [135]:
from transformers import BertModel
import torch
from tqdm.auto import tqdm

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
model = BertModel.from_pretrained("bert-base-cased")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [136]:
for i, ex in enumerate(combined):
    for key in ["input_ids", "attention_mask", "token_type_ids"]:
        if ex.get(key) is None:
            print(f"Пропущено значение в примере {i}: {key} = None")

In [137]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import numpy as np

import data_prep
from data_prep import PadSequence

collate_fn = PadSequence(['input_ids','token_type_ids','attention_mask'])

dataloader = torch.utils.data.DataLoader(
    combined,
    batch_size=32,
    shuffle=False,
    collate_fn=collate_fn
)

batch = next(iter(dataloader))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_embeddings = []

batch

for batch in tqdm(dataloader, desc="Extracting embeddings"):
    batch = {k: batch[k].to(device) for k in ['input_ids','token_type_ids','attention_mask']}
    with torch.no_grad():
        outputs=model(**batch)
        cls_emb = model(**batch).last_hidden_state[:, 0] # (batch_size, emb_size=768)
    all_embeddings.append(cls_emb.cpu())

all_embeddings = torch.cat(all_embeddings).numpy()

Extracting embeddings:   0%|          | 0/15732 [00:00<?, ?it/s]

In [138]:
combined = combined.add_column("cls_embedding", all_embeddings.tolist())

In [139]:
combined

Dataset({
    features: ['tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'domain_label', 'cls_embedding'],
    num_rows: 503394
})

In [140]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
wnut_scores = np.zeros(len(combined))
X = combined["cls_embedding"].cpu().numpy()
Y = combined["domain_label"].cpu().numpy()

indices = np.arange(len(combined))
for i, (train_idxs, test_idxs) in enumerate(tqdm(kf.split(X, Y), total=k, desc="Training classifiers")):
    X_train, X_test = X[train_idxs], X[test_idxs]
    Y_train = Y[train_idxs]
    
    clf = LogisticRegression(max_iter=1000, class_weight="balanced")
    clf.fit(X_train, Y_train)
    probas = clf.predict_proba(X_test)[:, 1]
    
    wnut_scores[test_idxs] = probas
    
combined = combined.add_column("wnut_score", wnut_scores.tolist())

Training classifiers:   0%|          | 0/5 [00:00<?, ?it/s]

In [141]:
# отбираем топ 20 % объектов из reddit_dataset по схожести с датасетом с wnut

reddit_subset = combined.filter(lambda x: x["domain_label"] == 0)

reddit_sorted = reddit_subset.sort("wnut_score", reverse=True) 

# построить гистограмму с wnut_score

top_20_count = int(0.20 * len(reddit_sorted))
top_20_reddit = reddit_sorted.select(range(top_20_count))

# top_20_reddit = top_20_reddit.remove_columns([
#     col for col in top_20_reddit.column_names if col not in ['input_ids', 'token_type_ids', 'attention_mask']
# ])

Filter:   0%|          | 0/503394 [00:00<?, ? examples/s]

In [142]:
sel_examples = top_20_reddit['tokens']

import json
with open("sel_examples.json", "w", encoding="utf-8") as f:
    json.dump(sel_examples, f, ensure_ascii=False, indent=2)

Теперь есть топ 20 % из 500 000 примеров для файн-тюна самого BERTа на задаче masked LM.

Код для файн-тюнинга BERTа на задаче MLM лежит в ноутбуке mlm. 

In [51]:
import datasets
import typing as tp
import inspect
import data_prep
import numpy as np
import sklearn
import torch 
from transformers import (pipeline, 
        AutoModelForTokenClassification, AutoTokenizer, 
        BertForTokenClassification, BertTokenizer)
from datasets import load_dataset, load_from_disk
import torch
from tqdm.auto import tqdm
from transformers import AdamW, BertTokenizer, BertForMaskedLM, set_seed, BertTokenizerFast
import pandas as pd
import warnings

import data_prep
import importlib
importlib.reload(data_prep)

# conll = load_dataset("datasets/conll2003")
# wnut = load_dataset("datasets/wnut_17")

conll = datasets.load_dataset("conll2003") # DatasetDict
wnut = datasets.load_dataset("wnut_17")

CONLL_NER_TAGS = conll['train'].features['ner_tags'].feature.names # всего 9 тегов
WNUT_NER_TAGS = wnut['train'].features['ner_tags'].feature.names

label_mapping = {
    'O': 'O',
    'B-location': 'B-LOC',
    'I-location': 'I-LOC',
    'B-group': 'B-ORG',
    'B-corporation': 'B-ORG',
    'B-person': 'B-PER',
    'B-creative-work': 'B-MISC',
    'B-product': 'B-MISC',
    'I-person': 'I-PER',
    'I-creative-work': 'I-MISC',
    'I-corporation': 'I-ORG',
    'I-group': 'I-ORG',
    'I-product': 'I-MISC'
}

labelindexmapping = {WNUT_NER_TAGS.index(k):CONLL_NER_TAGS.index(v) for k, v in label_mapping.items()}

converted_wnut = wnut.map(lambda x: data_prep.convert_label_sequence(x, labelindexmapping))

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model_name = "dslim/bert-base-NER"
# model = AutoModelForTokenClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

model_name = "model_base"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

set_seed(42)

label2id_conll= {'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8
}

id2label_conll = {v : k for k, v in label2id_conll.items()}

test_sentence = "His name is Jerry Abrahamson"
test_example = {"tokens": test_sentence.split(" "), "ner_tags": [0, 0, 0, 1, 2]}
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)

assert tokenizer.decode(test_result['input_ids']) == '[CLS] His name is Jerry Abrahamson [SEP]'

                                     #CLS     His  name is    Jerry    Abraham   ##son      SEP
assert test_result['text_labels'] == ['O'] + ["O", "O", "O", "B-PER", "I-PER",  "I-PER"] + ["O"]

conll = conll.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

wnut = converted_wnut
wnut = wnut.map(lambda x: data_prep.tokenize_and_preserve_tags(x, tokenizer, label2id_conll))

conll.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)
wnut.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'], output_all_columns=True)

input_keys = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
batch_size = 16
wnut_train_dataloader = torch.utils.data.DataLoader(wnut["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
wnut_test_dataloader = torch.utils.data.DataLoader(wnut["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_train_dataloader = torch.utils.data.DataLoader(conll["train"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))
conll_test_dataloader = torch.utils.data.DataLoader(conll["test"], 
                                                    batch_size=batch_size, 
                                                    collate_fn=data_prep.PadSequence(input_keys))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at model_base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
counter = {k : 0 for k in label2id_conll.keys()}
for example in wnut['train']['text_labels']:
    for label in example:
        counter[label] += 1

total=0
for count in counter.values():
    total+=count

freqs = {
    "O" : counter['O']/total*100,
    "PER" : (counter['B-PER']+counter['I-PER'])/total*100,
    "ORG" : (counter['B-ORG']+counter['I-ORG'])/total*100,
    "LOC" : (counter['B-LOC']+counter['I-LOC'])/total*100,
    "MISC" : (counter['B-MISC']+counter['I-MISC'])/total*100  
}

freqs = {k : round(v,3) for k, v in freqs.items()}

freqs

{'O': 95.293, 'PER': 1.485, 'ORG': 1.068, 'LOC': 1.125, 'MISC': 1.028}

In [53]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, get_scheduler, BertTokenizerFast
import torch
from torch.nn.utils import clip_grad_norm_ # выполняет обрезку градиентов для избежания их взрыва при обучении
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

import importlib
import data_prep
import model_utils
importlib.reload(data_prep)
importlib.reload(model_utils)

from model_utils import train_eval_ner

Если использовать continued pre-trained модель, полученную в ноутбуке mlm

In [56]:
label2id_conll

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [61]:
from transformers import BertTokenizerFast, BertForMaskedLM, AutoModelForTokenClassification

model_name = "dslim/bert-base-NER"
mlm_model = BertForMaskedLM.from_pretrained("model")
pretrained_bert = mlm_model.bert
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
model_ft.bert = pretrained_bert
# model_ft.classifier.reset_parameters()

tokenizer = AutoTokenizer.from_pretrained("model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for name, param in model_ft.bert.named_parameters():
    param.requires_grad = False

trainable_params = filter(lambda p: p.requires_grad, model_ft.parameters())
optimizer = torch.optim.Adam(trainable_params, lr=5e-4, betas=(0.9, 0.999), eps=1e-06)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = wnut['train']
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 16
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="constant",
    optimizer=optimizer
)

# добавляем веса
factor = 1.0
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)

fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 39.48it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:03<00:00, 53.75it/s]


...

Epoch: 8	 Train: 0.13756745501897985 	 Test: 0.28678881533351946
F1: 0.24791265253693
LR: [0.0005]



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 213/213 [00:03<00:00, 53.88it/s]


Epoch: 9	 Train: 0.13574842920640545 	 Test: 0.2866373121002574
F1: 0.2482495225970719
LR: [0.0005]



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 213/213 [00:03<00:00, 53.84it/s]


KeyboardInterrupt: 

In [8]:
model_ft

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [70]:
from transformers import BertTokenizerFast, BertForMaskedLM, AutoModelForTokenClassification

model_name = "dslim/bert-base-NER"
mlm_model = BertForMaskedLM.from_pretrained("model")
pretrained_bert = mlm_model.bert
model_ft = AutoModelForTokenClassification.from_pretrained(model_name)
model_ft.bert = pretrained_bert
model_ft.classifier.reset_parameters()

tokenizer = AutoTokenizer.from_pretrained("model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for p in model_ft.bert.parameters():           
    p.requires_grad = False
for n, p in model_ft.bert.named_parameters():
    if n.startswith(("encoder.layer.8","encoder.layer.9","encoder.layer.10", "encoder.layer.11")):
        p.requires_grad = True
for p in model_ft.classifier.parameters():     
    p.requires_grad = True

head_params = [p for n, p in model_ft.named_parameters() if "classifier" in n]
bert_params = [p for n, p in model_ft.named_parameters()
               if p.requires_grad and "classifier" not in n]

optimizer = torch.optim.AdamW(
    [
        {"params": head_params, "lr": 1e-4, "weight_decay": 0.01},
        {"params": bert_params, "lr": 2e-5, "weight_decay": 0.01},
    ],
    betas=(0.9, 0.999), eps=1e-6
)

num_epochs = 10
warmup_factor = 0.1

ft_train_dataset = wnut['train']
ft_test_dataset = wnut['test']
base_test_dataset = conll['test']

batch_size = 32
batches_per_epoch= len(ft_train_dataset) // batch_size if (len(ft_train_dataset) % batch_size)==0 \
                                                        else len(ft_train_dataset) // batch_size + 1
num_training_steps = num_epochs * batches_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(warmup_factor * num_training_steps),
    num_training_steps=num_training_steps,
)

factor = 0.01
num_labels = 9
class_weights = torch.tensor([factor if i == 0 else 1.0 for i in range(num_labels)], device=device)
loss_fn = CrossEntropyLoss(weight=class_weights)

fine_tuned_model, ft_report_before, ft_report_after, ft_metrics_before, ft_metrics_after = train_eval_ner(
    model_ft, tokenizer, device, optimizer, num_epochs, lr_scheduler, loss_fn,
    ft_train_dataset, ft_test_dataset, base_test_dataset, batch_size,
    id2label_conll
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Evaluating model without fine-tuning


100%|███████████████████████████████████████████████████████████████████████████████████| 41/41 [00:02<00:00, 19.31it/s]



Fine-tuning model


Epoch 1: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:06<00:00, 15.55it/s]


...

Epoch: 8	 Train: 0.3556981022582851 	 Test: 0.3777718638501516
F1: 0.22210414452709887
LR: [2.2222222222222223e-05, 4.444444444444444e-06]



Epoch 9: 100%|████████████████████████████████████████████████████████████████████████| 107/107 [00:06<00:00, 15.47it/s]


Epoch: 9	 Train: 0.40477499656518484 	 Test: 0.40453814260843324
F1: 0.2043939767958529
LR: [1.1111111111111112e-05, 2.222222222222222e-06]



Epoch 10: 100%|███████████████████████████████████████████████████████████████████████| 107/107 [00:06<00:00, 15.44it/s]


Epoch: 10	 Train: 0.37857362198008 	 Test: 0.3966017576252542
F1: 0.21237585943468298
LR: [0.0, 0.0]

                   pr_base      rec_base       f1_base         pr_ft  \
LOC           0.56 (+0.55)  0.89 (+0.77)  0.69 (+0.68)  0.26 (+0.26)   
MISC          0.06 (+0.06)  0.07 (+0.06)  0.06 (+0.05)  0.06 (+0.06)   
ORG           0.30 (+0.28)  0.70 (+0.62)  0.42 (+0.39)  0.09 (+0.09)   
PER           0.46 (+0.46)  0.79 (+0.77)  0.58 (+0.58)  0.30 (+0.30)   
micro avg     0.39 (+0.38)  0.70 (+0.63)  0.50 (+0.49)  0.15 (+0.15)   
macro avg     0.34 (+0.33)  0.61 (+0.55)  0.44 (+0.43)  0.18 (+0.18)   
weighted avg  0.39 (+0.38)  0.70 (+0.63)  0.50 (+0.49)  0.19 (+0.19)   

                    rec_ft         f1_ft  
LOC           0.51 (+0.39)  0.34 (+0.34)  
MISC          0.21 (+0.21)  0.09 (+0.09)  
ORG           0.36 (+0.30)  0.14 (+0.13)  
PER           0.47 (+0.46)  0.37 (+0.37)  
micro avg     0.39 (+0.35)  0.21 (+0.21)  
macro avg     0.39 (+0.34)  0.24 (+0.24)  
weighted avg  0.39 (

In [71]:
fine_tuned_model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [72]:
id2label_conll

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [73]:
test_sentences = [
    {'text' : 'His name is Jerry Abrahamson', 'ner_tags': [0, 0, 0, 1, 2]},
    {'text' : 'London is big city', 'ner_tags' : [5, 0, 0, 0]},
    {'text' : 'Jason Statham is a famous actor from America', 'ner_tags' : [1, 2, 0, 0, 0, 0, 0, 5]}
]

In [74]:
idx = 2
test_sentence = test_sentences[idx]['text']

tokenizer(test_sentence)

{'input_ids': [101, 3627, 1457, 9779, 2312, 1110, 170, 2505, 2811, 1121, 1738, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [75]:
fine_tuned_model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [76]:
id2label_conll

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

##### Тест NER

In [77]:
model_test = AutoModelForTokenClassification.from_pretrained("model_base")
model_test.to(device)
tokenizer_test = AutoTokenizer.from_pretrained("model_base")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at model_base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [78]:
import importlib
import data_prep
importlib.reload(data_prep)

idx = 2
test_sentence = test_sentences[idx]['text']
ner_tags = test_sentences[idx]['ner_tags']
test_example = {"tokens": test_sentence.split(" "), "ner_tags": ner_tags}
test_result = data_prep.tokenize_and_preserve_tags(test_example, tokenizer, label2id_conll)
print(test_result.word_ids())
print(test_result['text_labels'])
print(test_result['input_ids'])
print([tokenizer.decode(token) for token in test_result['input_ids']])

test_batch = {k : [v] for k, v in test_result.items()}

import torch
from collections import defaultdict

batched_test_batch = defaultdict(list)

for key, value in test_batch.items():
    try:
        batched_test_batch[key] = torch.tensor(value)
    except ValueError:
        batched_test_batch[key] = value

for key in batched_test_batch:
    if isinstance(batched_test_batch[key], torch.Tensor):
        batched_test_batch[key] = batched_test_batch[key].to(device)

import importlib
import model_utils
importlib.reload(model_utils)

from model_utils import NamedEntityPredictor

print(f"Initial model")
ner_test = NamedEntityPredictor(model_test, tokenizer, id2label_conll)
dt=ner_test.predict(batched_test_batch)
print(dt["predicted_labels"])
print(f"")

print(f"Fine-tuned model")
ner_test = NamedEntityPredictor(fine_tuned_model, tokenizer, id2label_conll)
dt=ner_test.predict(batched_test_batch)
print(dt["predicted_labels"])

# print(f"Predicted:\t {ner_test.predict(batched_test_batch)['predicted_labels']}")
# print(f"True:\t {batched_test_batch['text_labels']}")

pred = ner_test.predict(batched_test_batch)['predicted_labels']
true = batched_test_batch['text_labels']
for pred_seq, true_seq in zip(pred, true):
    aligned = [(t, p) for t, p in zip(true_seq, pred_seq)]
    for t, p in aligned:
        print(f"True: {t:<10}  Pred: {p:<10}")


[None, 0, 1, 1, 1, 2, 3, 4, 5, 6, 7, None]
['O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O']
[101, 3627, 1457, 9779, 2312, 1110, 170, 2505, 2811, 1121, 1738, 102]
['[CLS]', 'Jason', 'St', '##ath', '##am', 'is', 'a', 'famous', 'actor', 'from', 'America', '[SEP]']
Initial model
[['O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-PER', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'B-LOC', 'I-LOC']]

Fine-tuned model
[['O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O']]
True: O           Pred: O         
True: B-PER       Pred: B-PER     
True: I-PER       Pred: I-PER     
True: I-PER       Pred: I-PER     
True: I-PER       Pred: I-PER     
True: O           Pred: O         
True: O           Pred: O         
True: O           Pred: O         
True: O           Pred: O         
True: O           Pred: O         
True: B-LOC       Pred: B-LOC     
True: O           Pred: O         
